# Libraries

In [ ]:
import cv2
import numpy as np
import json
import os

# Functions

In [ ]:
def exclude_cont_bound(myconts: dict, n_x: float, n_y: float) -> dict:
    """
    Exclude contours that touch the boundary of the image.
    
    :param myconts: Dictionary of contours
    :param n_x: Width of the image or criterion for exclusion
    :param n_y: Height of the image or criterion for exclusion

    :return: Filtered dictionary of contours that do not touch the boundary
    """
    
    myconts_copy = myconts.copy()
    for i in myconts:
        i_contour = myconts[i]
        for j in range(len(i_contour)):
            x, y = i_contour[j][0]
            if x == 0 or y == 0 or x >= n_x or y >= n_y:
                save_or_not_save = False
                break
        else:
            save_or_not_save = True
        if save_or_not_save == False:
          del myconts_copy[i]
          
    return myconts_copy


def extract_contours(contours: list) -> dict: # Luiz check this type hint. I think it is correct but you should check it.
    """
    Extract contours from a list of contours and return them as a dictionary.
    
    :param contours: Contours to be processed
    
    :return: Contour points. Each index of the dictionary corresponds to a contour.
    """

    myconts = {}
    for i, contour in enumerate(contours):
        myconts[str(i)] = np.array(contour)
        
    return myconts

# Image or images

In [ ]:
filename = 'dataset/binary-masks/01.png' # Commen if you want to use a set of images
filepath = [filename] # Luiz build a loop to read all images in a folder

# Get contour

In [ ]:
for filename in filepath:
    # Read image
    img = cv2.imread(filename)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, threshold = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    # Get contour
    myconts = extract_contours(contours)

    # Exclude contours that touch the boundary
    myc = exclude_cont_bound(myconts, 2499, 2499)

    # Create a dictionary to save contours in JSON format
    contours_json = {}
    for idx, contour in myc.items():
        coords = contour[:, 0, :]
        contours_json[f"{idx}"] = {
            'x': coords[:, 0].tolist(),
            'y': coords[:, 1].tolist(),
            'q': "plot here a pixel_aggregate/pixel_full_image rate"
        }

# Save contours into a JSON file
with open('dataset_contours_aggregate.json', 'w') as f:
    json.dump(contours_json, f, indent=4)

# Plot the dataset

In [ ]:
# Create a function to select one contour in dataset and plot it
# Put this function on the cell "# Functions" (see above)